In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
import os
repo_path = Path.cwd()/'drive/MyDrive/calcification_detection/calc-det/notebooks/'
os.chdir(str(repo_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/calcification_detection/data_rois_128.zip /home/data_rois.zip
!unzip /home/data_rois.zip -d /home
!pip install transformers

Streaming output truncated to the last 5000 lines.
  inflating: /home/data_rois/patches/53586751/53586751_roi_94.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_96.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_98.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_101.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_102.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_103.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_104.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_106.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_107.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_108.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_109.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_110.png  
  inflating: /home/data_rois/patches/53586751/53586751_roi_111.png  
  inflating: /home/data_rois/patches/53586751/53586751_

In [ ]:
from pathlib import Path
thispath = Path.cwd().resolve()
import yaml

In [ ]:
# This code is an adapted version of Pytorch's ResNet Implementation
import torch
import numpy as np

from collections import OrderedDict
from typing import Type, Callable, Union, List, Optional
from torch import Tensor, nn


def conv3x3(
    in_planes: int, out_planes: int, stride: int = 1,
    groups: int = 1, dilation: int = 1
):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes, out_planes, kernel_size=3, stride=stride,
        padding=dilation, groups=groups, bias=False, dilation=dilation
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1):
    """1x1 convolution"""
    return nn.Conv2d(
        in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1
    
    def __init__(
        self, inplanes: int, planes: int, stride: int = 1,
        downsample: Optional[nn.Module] = None, groups: int = 1,
        base_width: int = 64, dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        act_fn: Optional[Callable[..., nn.Module]] = None,
        use_middle_act: bool = True
    ):
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        if act_fn is None:
            act_fn = nn.ReLU

        self.use_middle_activation = use_middle_act

        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.act_fn = act_fn()
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        if self.use_middle_activation:
            out = self.act_fn(out)

        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.act_fn(out)
        return out


class Bottleneck(nn.Module):
    expansion: int = 4

    def __init__(
        self,
        inplanes: int, planes: int, stride: int = 1,
        downsample: Optional[nn.Module] = None, groups: int = 1,
        base_width: int = 64, dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        act_fn: Optional[Callable[..., nn.Module]] = None
    ):
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if act_fn is None:
            act_fn = nn.ReLU
        width = int(planes * (base_width / 64.0)) * groups

        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.act_fn = act_fn()
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act_fn(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.act_fn(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.act_fn(out)

        return out


class ResNetBased(nn.Module):
    def __init__(
        self,
        block: str = 'basic',
        # layers: List[int],
        num_classes: int = 1,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        inplanes: int = 64,
        act_fn: Optional[Callable[..., nn.Module]] = None,
        downsample_blocks: int = 3,
        fc_dims: np.ndarray = None,
        dropout: float = 0,
        use_middle_act: bool = True,
        block_act: Optional[Callable[..., nn.Module]] = None
    ):
        super().__init__()

        if block == 'basic':
            block = BasicBlock
        else:
            block = Bottleneck

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        self.use_middle_act = use_middle_act

        if act_fn is None:
            act_fn = nn.ReLU
        if block_act is None:
            block_act = nn.ReLU
        self.block_act = block_act

        self.inplanes = inplanes
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )

        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.act_fn = act_fn

        self.inner_layers = []
        out_planes = self.inplanes
        for i in range(downsample_blocks):
            if i == 0:
                self.inner_layers.append(
                    (f'layer{i+1}', self._make_layer(block, out_planes * 2, 2))
                )
            else:
                self.inner_layers.append(
                    (f'layer{i+1}',
                     self._make_layer(
                        block, out_planes * 2, 2, stride=2,
                        dilate=replace_stride_with_dilation[0])))
            out_planes = out_planes * 2

        self.inner_layers = nn.Sequential(OrderedDict(self.inner_layers))

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.in_features = out_planes
        if fc_dims is not None:
            layers_list = []
            for i in range(len(fc_dims)):
                in_neurons = self.in_features if i == 0 else fc_dims[i-1]
                layers_list = layers_list + [
                    (f'do{i+1}', nn.Dropout(dropout)),
                    (f'fc{i+1}', nn.Linear(in_neurons, fc_dims[i])),
                    (f'act{i+1}', self.act_fn())]
            layers_list.append((f'fc{i+2}', nn.Linear(fc_dims[i], 1)))
            self.classifier = nn.Sequential(OrderedDict(layers_list))
        else:
            self.classifier = nn.Sequential(OrderedDict([
                ('do', nn.Dropout(dropout)),
                ('fc', nn.Linear(self.in_features, num_classes))
            ]))

        if isinstance(act_fn, nn.LeakyReLU):
            nl = 'leaky_relu'
        else:
            nl = 'relu'

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity=nl)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)
                nn.init.constant_(m.bias.data, 0)

    def _make_layer(
        self, block: Type[Union[BasicBlock, Bottleneck]], planes: int,
        blocks: int, stride: int = 1, dilate: bool = False,
    ):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation

        # Here we decide whether to use dilation convs or strides convs
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups,
                self.base_width, previous_dilation, norm_layer,
                act_fn=self.block_act, use_middle_act=self.use_middle_act
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                    act_fn=self.block_act,
                    use_middle_act=self.use_middle_act
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act_fn()(x)
        x = self.inner_layers(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [ ]:
from pathlib import Path
thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

from deep_learning.dataset.dataset import INBreast_Dataset_pytorch
from deep_learning.classification_models.models.base_classifier import CNNClasssifier
from deep_learning.classification_models.models.resnet_based_classifier import ResNetBased
import deep_learning.dl_utils as dl_utils

import logging
import torch
import time
import random
import yaml

import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from transformers import SwinForImageClassification

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def identity_function(arg):
    return arg


def train_model(datasets, dataloaders, data_transforms, model, criterion, optimizer, scheduler, cfg):

    # guarantee reproducibility
    since = time.time()
    random.seed(0)
    torch.manual_seed(1442)
    np.random.seed(0)

    # holders for best model
    best_metric = 0.0
    best_epoch = 0
    best_avgpr = 0
    best_f1 = 0
    last_three_losses = []
    early_stopping_count = 0
    previous_metric = 0
    previous_mean_loss = 0
    best_metric_name = cfg['training']['best_metric']

    exp_path = Path.cwd().parent.parent/f'data/deepl_runs/{cfg["experiment_name"]}'
    exp_path.mkdir(exist_ok=True, parents=True)
    best_model_path = exp_path / f'{cfg["experiment_name"]}_{best_metric_name}.pt'
    best_model_path_auroc = exp_path / f'{cfg["experiment_name"]}_auroc.pt'
    chkpt_path = exp_path / f'{cfg["experiment_name"]}_chkpt.pt'
    logging.info(f'Storing experiment in: {exp_path}')

    if cfg['training']['resume_training']:
        checkpoint = torch.load(chkpt_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        init_epoch = checkpoint['epoch'] + 1
    else:
        init_epoch = 0

    # tensorboard loggs
    log_dir = exp_path/'tensorboard'
    log_dir.mkdir(exist_ok=True, parents=True)
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(init_epoch, cfg['training']['n_epochs']):
        logging.info(f'Epoch {epoch+1}/{cfg["training"]["n_epochs"]}')
        logging.info(('-' * 10))

        # resample used negatives to use the large diversity of them that we have
        datasets['train'].update_sample_used(epoch)
        dataloaders['train'] = DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, num_workers=4, drop_last=False)

        for phase in ['train', 'val']:
            # Set model to the corresponding mode and update lr if necessary
            if phase == 'train':
                if epoch != 0:
                    if cfg['training']['lr_scheduler'] == 'ReduceLROnPlateau':
                        scheduler.step(previous_loss)
                    else:
                        scheduler.step()
                writer.add_scalar(f"LearningRate/{phase}", optimizer.param_groups[0]['lr'], epoch)
                model.train()
            else:
                model.eval()

            # define holders for losses, preds and labels
            running_loss = 0.0
            epoch_preds, epoch_labels = [], []

            # Iterate over data.
            if (cfg['training']['max_iters_per_epoch'] is not None) and (phase == 'train'):
                total_its = cfg['training']['max_iters_per_epoch']
            else:
                total_its = len(dataloaders[phase])

            for it, sample in tqdm(enumerate(dataloaders[phase]), total=total_its):
                # Apply transformations and send to device
                sample['img'] = data_transforms[phase](sample['img'])
                inputs = sample['img'].to(device)
                labels = sample['label'].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward pass (track history if only in train)
                with torch.set_grad_enabled(phase == 'train'):
                    # predict
                    outputs = model(inputs)
                    
                    if 'transformer' in cfg['model']['backbone']:
                        outputs = outputs['logits']
                    # store values
                    epoch_preds.append(np.asarray(
                        torch.sigmoid(outputs.detach()).flatten().cpu()))
                    epoch_labels.append(np.asarray(labels.detach().cpu()))

                    # finish the comp. graph
                    loss = criterion(outputs.flatten(), labels.float())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # once in a while store the images batch to check
                # if it in [100]:
                #     imgs = T.functional.rgb_to_grayscale(sample['img']).cpu()
                #     writer.add_images(f'Images/{phase}', imgs, epoch)
                #     del imgs

                # get the epoch loss cumulatively
                running_loss += loss.item() * inputs.size(0)

                if phase == 'train':
                    if (it != 0) and ((it % cfg['training']['log_iters']) == 0):
                        # compute and log the metrics for the iteration
                        iter_preds = np.concatenate(epoch_preds)
                        iter_labels = np.concatenate(epoch_labels)
                        iter_loss = running_loss / len(iter_preds)
                        iter_metrics = dl_utils.get_metrics(iter_labels, iter_preds)
                        dl_utils.tensorboard_logs(
                            writer, iter_loss, it+(total_its*epoch), iter_metrics, phase, True)
                    if cfg['training']['max_iters_per_epoch'] is not None:
                        if it == cfg['training']['max_iters_per_epoch']:
                            break

            # compute and log the metrics for the epoch
            epoch_preds = np.concatenate(epoch_preds)
            epoch_labels = np.concatenate(epoch_labels)
            epoch_loss = running_loss / len(epoch_preds)
            last_three_losses.append(epoch_loss)
            if len(last_three_losses) > 3:
                last_three_losses = last_three_losses[1:]
            metrics = dl_utils.get_metrics(epoch_labels, epoch_preds)
            dl_utils.tensorboard_logs(writer, epoch_loss, epoch, metrics, phase)

            # print status
            epoch_f1 = metrics['f1_score']
            message = f'{phase} Loss: {epoch_loss:.4f} Acc: {metrics["accuracy"]:.4f}' \
                      f' F1: {epoch_f1:.4f} AUROC: {metrics["auroc"]:.4f} AvgPR: {metrics["avgpr"]:.4f}'  
            logging.info(message)

            # save last and best checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss}, chkpt_path)

            if phase == 'val':
                if metrics[best_metric_name] > best_metric:
                    best_metric = metrics[best_metric_name]
                    best_f1 = epoch_f1
                    best_epoch = epoch+1
                    best_avgpr = metrics['avgpr']
                    best_threshold = metrics['threshold']
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'metrics': metrics,
                        'configuration': cfg
                        }, best_model_path)

                if cfg['training']['early_stopping'] and (epoch != 0):
                    diff = np.mean(last_three_losses) - previous_mean_loss
                    if -diff < cfg['training']['early_stopping_args']['min_diff']:
                        early_stopping_count += 1
                    else:
                        early_stopping_count = 0
                previous_mean_loss = np.mean(last_three_losses)
                previous_metric = metrics[best_metric_name]

        if cfg['training']['early_stopping']:
            max_epochs = cfg['training']['early_stopping_args']['max_epoch']
            if early_stopping_count == max_epochs:
                msg = f'Early stopping after {max_epochs} epochs without' \
                    f' significant change in val metric'
                logging.info(msg)
                break
        logging.info(('-' * 10))

    time_elapsed = time.time() - since
    message = f'Training complete in {(time_elapsed // 60):.0f}m ' \
        f'{(time_elapsed % 60):.0f}s'
    logging.info(message)
    logging.info(f'Best val {best_metric_name}: {best_metric:4f}, avgPR {best_avgpr:.4f},' \
                 f'threshold {best_threshold:.4f}, f1 {best_f1:.4f}, epoch {best_epoch}')

    # close the tensorboard session
    writer.flush()
    writer.close()

    # load best model weights before returning
    best_model = torch.load(best_model_path)
    model.load_state_dict(best_model['model_state_dict'])
    return model


def main():
    # read the configuration file
    config_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
    with open(config_path, "r") as ymlfile:
        cfg = yaml.safe_load(ymlfile)

    # use the configuration for the dataset
    dataset_arguments = cfg['dataset']
    dataset_arguments['patch_images_path'] = Path(dataset_arguments['patch_images_path'])
    datasets = {
        'train': INBreast_Dataset_pytorch(
            partitions=['train'], neg_to_pos_ratio=dataset_arguments['train_neg_to_pos_ratio'],
            balancing_seed=0, **dataset_arguments),
        'val': INBreast_Dataset_pytorch(
            partitions=['validation'], neg_to_pos_ratio=None, **dataset_arguments)
    }

    # use the configuration for the dataloaders
    dataloaders = {
        'val': DataLoader(
            datasets['val'], batch_size=cfg['dataloaders']['val_batch_size'],
            num_workers=4, drop_last=False),
        'train': DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, num_workers=4, drop_last=False)
    }

    # use the configuration for the transformations
    transforms = nn.Sequential(
        T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0, hue=0),
        T.RandomAffine(
            degrees=(0, 20), translate=None, scale=None, shear=(1, 10, 1, 10),
            interpolation=T.InterpolationMode.BILINEAR, fill=0
        ),
        T.RandomPerspective(distortion_scale=0.2),
        T.RandomRotation(degrees=(0, 20)),
        T.RandomRotation(degrees=(90, 110)),
        T.RandomResizedCrop(
            size=(cfg['model']['img_size'], cfg['model']['img_size']),
            scale=(0.9, 1), ratio=(1, 1)),
        T.RandomAutocontrast(),
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip()
    )
    transforms = T.RandomApply(transforms=transforms, p=cfg['data_aug']['prob'])
    data_transforms = {
        'train': identity_function if (cfg['data_aug']['prob'] == 0) else transforms,
        'val': identity_function
    }

    # model configs
    if cfg['model']['backbone'] == 'swin_transformer':
        model = SwinForImageClassification.from_pretrained(
            'microsoft/swin-tiny-patch4-window7-224',
            # num_labels=2,
            # id2label={str(0): 'normal', str(1): 'abnormal'},
            # label2id={'normal': str(0), 'abnormal': str(1)},
            num_labels=1,
            # id2label={str(1): 'abnormal'},
            # label2id={'abnormal': str(1)},
            ignore_mismatched_sizes = True,
        )
    elif cfg['model']['backbone'] == 'vanilla':
        model = VanillaCNN(
            dropout=cfg['model']['dropout'],
            activation=getattr(nn, cfg['model']['activation'])()
        )
    elif cfg['model']['backbone'] == 'net1':
        model = Net1(
            dropout=cfg['model']['dropout'],
            activation=getattr(nn, cfg['model']['activation'])(inplace=True),
            n_downsamples=cfg['model']['n_downsamples'],
            fc_dims=cfg['model']['fc_dims'],
            image_size=cfg['model']['img_size']
        )
    elif cfg['model']['backbone'] == 'net2':
        model = ResNetBased(
            block=cfg['model']['block'],
            replace_stride_with_dilation=cfg['model']['replace_stride_with_dilation'],
            inplanes=cfg['model']['inplanes'],
            act_fn=getattr(nn, cfg['model']['activation']),
            downsample_blocks=cfg['model']['n_downsamples'],
            fc_dims=cfg['model']['fc_dims'],
            dropout=cfg['model']['dropout'],
            use_middle_act=cfg['model']['use_middle_activation'],
            block_act=getattr(nn, cfg['model']['bloc_act']),
        )
    else:
        model = CNNClasssifier(
            activation=getattr(nn, cfg['model']['activation'])(),
            dropout=cfg['model']['dropout'],
            fc_dims=cfg['model']['fc_dims'],
            freeze_weights=cfg['model']['freeze_weights'],
            backbone=cfg['model']['backbone'],
            pretrained=cfg['model']['pretrained'],
        )
        model = model.model
    model = model.to(device)

    # training configs
    criterion = getattr(nn, cfg['training']['criterion'])()

    optimizer = getattr(optim, cfg['training']['optimizer'])
    optimizer = optimizer(model.parameters(), **cfg['training']['optimizer_args'])

    scheduler = getattr(lr_scheduler, cfg['training']['lr_scheduler'])
    scheduler = scheduler(optimizer, **cfg['training']['lr_scheduler_args'])

    # train the model
    train_model(datasets, dataloaders, data_transforms, model, criterion, optimizer, scheduler, cfg)

In [ ]:
cfg = {
    'model': {
        'activation': 'LeakyReLU',
        'bloc_act': None,
        'use_middle_activation': True,
        'dropout': 0.4,
        'fc_dims': None,
        'freeze_weights': False,
        'backbone': 'net2',
        'pretrained': True,
        'n_downsamples': 3,
        'img_size': 32,
        'block': 'basic',
        'replace_stride_with_dilation': None,
        'inplanes': 64
    },
    'dataset': {
        'extract_patches': False,
        'delete_previous': False,
        'extract_patches_method': 'centered',
        'patch_size': 128,
        'crop_size': 32,
        'center_noise': 10,
        'stride': 64,
        'min_breast_fraction_roi': 0.7,
        'n_jobs': -1,
        'cropped_imgs': True,
        'ignore_diameter_px': 15,
        'patch_images_path': '/home/data_rois/',
        'train_neg_to_pos_ratio': 10,   # 2% is the original ratio, 37 max
        'get_lesion_bboxes': True,
        'for_detection_net': False,
        'normalization': 'z_score'
    },
    'dataloaders': {
        'train_batch_size': 128,
        'val_batch_size': 256
    },
    'data_aug': {
        'prob': 0
    },
    'training': {
        'criterion': 'BCEWithLogitsLoss',
        'optimizer': 'Adam',
        'optimizer_args': {
            'lr': 0.0001 #, 'momentum': 0.9
        },
        'lr_scheduler': 'StepLR',
        'lr_scheduler_args': {
            'step_size': 7, 'gamma': 0.1
        },
        'n_epochs': 30,
        'best_metric': 'f1_score',
        'resume_training': False,
        'early_stopping': True,
        'early_stopping_args':{
            'min_diff': 0.0001,
            'max_epoch': 3
        },
        'log_iters': 100,
        'max_iters_per_epoch': None
        },
    'experiment_name': '32_resnet_01'
}

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)

In [ ]:
cfg['model']['activation'] = 'LeakyReLU'
cfg['model']['dropout'] = 0.4
cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'net2'
cfg['model']['n_downsamples'] = 3
cfg['model']['img_size'] = 32
cfg['model']['block'] = 'basic'
cfg['model']['replace_stride_with_dilation'] = None
cfg['model']['inplanes'] = 64
cfg['dataset']['crop_size'] = 32
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['dataset']['normalization'] = 'z_score'
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.5
cfg['training']['criterion'] = 'BCEWithLogitsLoss'
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 7, 'gamma': 0.1}
cfg['training']['n_epochs'] = 30
cfg['training']['early_stopping'] = True
cfg['training']['early_stopping_args'] = {'min_diff': 0.00001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'auroc'
cfg['training']['max_iters_per_epoch'] = 500
cfg['experiment_name'] = '32_net2_03'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:51: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/32_net2_03
INFO:root:Epoch 1/30
INFO:root:----------
 52%|█████▏    | 261/500 [00:46<00:42,  5.58it/s]
INFO:root:train Loss: 0.2171 Acc: 0.8002 F1: 0.3997 AUROC: 0.8482 AvgPR: 0.5722
100%|██████████| 205/205 [00:23<00:00,  8.54it/s]
INFO:root:val Loss: 0.3886 Acc: 0.8288 F1: 0.1050 AUROC: 0.9210 AvgPR: 0.1811
INFO:root:----------
INFO:root:Epoch 2/30
INFO:root:----------
 52%|█████▏    | 261/500 [00:46<00:42,  5.59it/s]
INFO:root:train Loss: 0.1483 Acc: 0.8980 F1: 0.5898 AUROC: 0.9257 AvgPR: 0.7653
100%|██████████| 205/205 [00:24<00:00,  8.39it/s]
INFO:root:val Loss: 0.0322 Acc: 0.9746 F1: 0.4438 AUROC: 0.9696 Av

In [ ]:
cfg['model']['activation'] = 'GELU'
cfg['model']['use_middle_activation'] = False
cfg['model']['dropout'] = 0.4
cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'net2'
cfg['model']['n_downsamples'] = 3
cfg['model']['img_size'] = 32
cfg['model']['block'] = 'basic'
cfg['model']['replace_stride_with_dilation'] = None
cfg['model']['inplanes'] = 64
cfg['dataset']['crop_size'] = 32
cfg['dataset']['center_noise'] = 5
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0
cfg['training']['criterion'] = 'BCEWithLogitsLoss'
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 7, 'gamma': 0.1}
cfg['training']['n_epochs'] = 30
cfg['training']['early_stopping'] = True
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'auroc'
cfg['training']['max_iters_per_epoch'] = 300
cfg['experiment_name'] = '32_net2_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:51: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/32_net2_04
INFO:root:Epoch 1/30
INFO:root:----------
 20%|█▉        | 59/300 [00:13<00:56,  4.29it/s]


KeyboardInterrupt: ignored

In [ ]:
cfg['model']['activation'] = 'GELU'
cfg['model']['use_middle_activation'] = False
cfg['model']['dropout'] = 0.4
cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'net2'
cfg['model']['n_downsamples'] = 3
cfg['model']['img_size'] = 32
cfg['model']['block'] = 'basic'
cfg['model']['replace_stride_with_dilation'] = None
cfg['model']['inplanes'] = 64
cfg['dataset']['crop_size'] = 32
cfg['dataset']['center_noise'] = 5
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0
cfg['training']['criterion'] = 'BCEWithLogitsLoss'
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 7, 'gamma': 0.1}
cfg['training']['n_epochs'] = 30
cfg['training']['early_stopping'] = True
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'auroc'
cfg['training']['max_iters_per_epoch'] = 300
cfg['experiment_name'] = '32_net2_05'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:51: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/32_net2_05
INFO:root:Epoch 1/30
INFO:root:----------
100%|██████████| 300/300 [01:08<00:00,  4.35it/s]
INFO:root:train Loss: 0.0775 Acc: 0.8472 F1: 0.2129 AUROC: 0.8876 AvgPR: 0.5432
100%|██████████| 205/205 [00:25<00:00,  7.98it/s]
INFO:root:val Loss: 0.0275 Acc: 0.9413 F1: 0.2655 AUROC: 0.9692 AvgPR: 0.7364
INFO:root:----------
INFO:root:Epoch 2/30
INFO:root:----------
100%|██████████| 300/300 [01:11<00:00,  4.18it/s]
INFO:root:train Loss: 0.0364 Acc: 0.9660 F1: 0.5749 AUROC: 0.9732 AvgPR: 0.8384
100%|██████████| 205/205 [00:26<00:00,  7.73it/s]
INFO:root:val Loss: 0.0183 Acc: 0.9687 F1: 0.4122 AUROC: 0.9886 Av

In [ ]:
cfg['model']['activation'] = 'GELU'
cfg['model']['use_middle_activation'] = True
cfg['model']['dropout'] = 0.4
cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'net2'
cfg['model']['n_downsamples'] = 3
cfg['model']['img_size'] = 32
cfg['model']['block'] = 'basic'
cfg['model']['replace_stride_with_dilation'] = None
cfg['model']['inplanes'] = 64
cfg['dataset']['crop_size'] = 32
cfg['dataset']['center_noise'] = 5
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0
cfg['training']['criterion'] = 'BCEWithLogitsLoss'
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 7, 'gamma': 0.1}
cfg['training']['n_epochs'] = 30
cfg['training']['early_stopping'] = True
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'auroc'
cfg['training']['max_iters_per_epoch'] = 300
cfg['experiment_name'] = '32_net2_05'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:51: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/32_net2_05
INFO:root:Epoch 1/30
INFO:root:----------
100%|██████████| 300/300 [01:16<00:00,  3.94it/s]
INFO:root:train Loss: 0.0707 Acc: 0.8623 F1: 0.2413 AUROC: 0.9150 AvgPR: 0.6092
100%|██████████| 205/205 [00:27<00:00,  7.46it/s]
INFO:root:val Loss: 0.0235 Acc: 0.9701 F1: 0.4120 AUROC: 0.9749 AvgPR: 0.7813
INFO:root:----------
INFO:root:Epoch 2/30
INFO:root:----------
100%|██████████| 300/300 [01:15<00:00,  3.96it/s]
INFO:root:train Loss: 0.0299 Acc: 0.9724 F1: 0.6323 AUROC: 0.9825 AvgPR: 0.8802
100%|██████████| 205/205 [00:27<00:00,  7.40it/s]
INFO:root:val Loss: 0.0191 Acc: 0.9710 F1: 0.4299 AUROC: 0.9887 Av

In [ ]:
cfg['model']['activation'] = 'GELU'
cfg['model']['bloc_act'] = 'GELU'
cfg['model']['use_middle_activation'] = True
cfg['model']['dropout'] = 0.4
cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'net2'
cfg['model']['n_downsamples'] = 3
cfg['model']['img_size'] = 32
cfg['model']['block'] = 'basic'
cfg['model']['replace_stride_with_dilation'] = None
cfg['model']['inplanes'] = 64
cfg['dataset']['crop_size'] = 32
cfg['dataset']['center_noise'] = 5
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0
cfg['training']['criterion'] = 'BCEWithLogitsLoss'
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 7, 'gamma': 0.1}
cfg['training']['n_epochs'] = 30
cfg['training']['early_stopping'] = True
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'auroc'
cfg['training']['max_iters_per_epoch'] = 300
cfg['experiment_name'] = '32_net2_06'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:51: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/32_net2_06
INFO:root:Epoch 1/30
INFO:root:----------
100%|██████████| 300/300 [01:15<00:00,  3.98it/s]
INFO:root:train Loss: 0.0776 Acc: 0.8686 F1: 0.2409 AUROC: 0.9044 AvgPR: 0.5598
100%|██████████| 205/205 [00:27<00:00,  7.54it/s]
INFO:root:val Loss: 0.0227 Acc: 0.9576 F1: 0.3398 AUROC: 0.9800 AvgPR: 0.7725
INFO:root:----------
INFO:root:Epoch 2/30
INFO:root:----------
100%|██████████| 300/300 [01:15<00:00,  3.99it/s]
INFO:root:train Loss: 0.0320 Acc: 0.9560 F1: 0.5205 AUROC: 0.9807 AvgPR: 0.8661
100%|██████████| 205/205 [00:27<00:00,  7.54it/s]
INFO:root:val Loss: 0.0195 Acc: 0.9719 F1: 0.4352 AUROC: 0.9860 Av